In [1]:
#runnning the simulator, run on cmd on carla folder

#CarlaUE4.exe -windowed -carla-server -benchmark -fps=10

'CarlaUE4.exe' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [1]:
import carla
import random
import time
#connect to carla
client = carla.Client('localhost', 2000)
client.set_timeout(20)
# get World, load a new map.
world = client.get_world()
#world = client.load_world('Town05_Opt', carla.MapLayer.Buildings | carla.MapLayer.ParkedVehicles)

In [9]:
# unload buildings
# world.unload_map_layer(carla.MapLayer.Buildings)

In [ ]:
## NO-rendering option.

# settings = world.get_settings()
# settings.no_rendering_mode = True
# world.apply_settings(settings)
# ...
# settings.no_rendering_mode = False
# world.apply_settings(settings)


# No-rendering mode: Unreal Engine does not render anything. Graphics are not computed. GPU based sensors return empty data.
# Off-screen mode: Unreal Engine is working as usual, rendering is computed but there is no display available. GPU based sensors return data.


#./CarlaUE4.sh -RenderOffScreen

In [7]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

# Set the spectator with an empty transform
spectator.set_transform(carla.Transform())
# This will set the spectator at the origin of the map, with 0 degrees
# pitch, yaw and roll - a good way to orient yourself in the map


In [24]:
## get spectator location.

t = world.get_spectator().get_transform()
# coordinate_str = "(x,y) = ({},{})".format(t.location.x, t.location.y)
coordinate_str = "(x,y,z) = ({},{},{})".format(t.location.x, t.location.y,t.location.z)
print (coordinate_str)

(x,y,z) = (-131.56546020507812,-47.225643157958984,2.6048898696899414)


In [27]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

In [28]:
for x in spawn_points:
    print(x)

Transform(Location(x=-44.188328, y=-39.610611, z=0.450000), Rotation(pitch=0.000000, yaw=-90.337494, roll=0.000000))
Transform(Location(x=-54.711468, y=-41.650139, z=0.450000), Rotation(pitch=0.000000, yaw=89.662514, roll=0.000000))
Transform(Location(x=-51.199158, y=-39.570789, z=0.450000), Rotation(pitch=0.000000, yaw=89.662514, roll=0.000000))
Transform(Location(x=-177.549835, y=2.981492, z=0.300000), Rotation(pitch=0.000000, yaw=-0.139465, roll=0.000000))
Transform(Location(x=-175.941330, y=6.477585, z=0.300000), Rotation(pitch=0.000000, yaw=-0.139465, roll=0.000000))
Transform(Location(x=-203.448898, y=-3.955487, z=0.300000), Rotation(pitch=0.000000, yaw=179.860489, roll=0.000000))
Transform(Location(x=-201.940384, y=-0.459150, z=0.300000), Rotation(pitch=0.000000, yaw=179.860489, roll=0.000000))
Transform(Location(x=-191.567139, y=14.351324, z=0.600000), Rotation(pitch=0.000000, yaw=89.984192, roll=0.000000))
Transform(Location(x=-184.574814, y=-13.591502, z=0.600000), Rotation(p

In [11]:
# ADD RANDOM VEHICLES ON THE MAP.

# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [2]:
# --------------
# Spawn ego vehicle
# --------------

# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

ego_location = carla.Transform(carla.Location(x=-131.6, y=-70.5, z=0.500000), 
                               carla.Rotation(pitch=0.000000, yaw=90, roll=0.000000))

ego_vehicle = world.spawn_actor(vehicle_blueprints.filter('model3')[0], ego_location)

# ego_bp = world.get_blueprint_library().find('vehicle.tesla.model3')
# ego_bp.set_attribute('role_name','ego')
# print('\nEgo role_name is set')
# ego_color = random.choice(ego_bp.get_attribute('color').recommended_values)
# ego_bp.set_attribute('color',ego_color)
# print('\nEgo color is set')

# spawn_points = world.get_map().get_spawn_points()
# number_of_spawn_points = len(spawn_points)

# if 0 < number_of_spawn_points:
#     random.shuffle(spawn_points)
#     ego_transform = spawn_points[0]
#     ego_vehicle = world.spawn_actor(ego_bp,ego_transform)
#     print('\nEgo is spawned')
# else: 
#     logging.warning('Could not found any spawn points')

In [3]:
# --------------
# Spawn attached RGB camera
# --------------
cam_bp = None
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(1920))
cam_bp.set_attribute("image_size_y",str(1080))
cam_bp.set_attribute("fov",str(105))
cam_location = carla.Location(2,0,1)
cam_rotation = carla.Rotation(0,180,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_cam.listen(lambda image: image.save_to_disk('tutorial/output/%.6d.jpg' % image.frame))

In [82]:
# # --------------
# # Spectator on ego cam position
# # --------------

# spectator = world.get_spectator()
# transform = ego_cam.get_transform()
# spectator.set_transform(carla.Transform(transform.location + carla.Location(z=0.5),
# carla.Rotation(yaw=90)))


In [3]:
# --------------
# Spectator on vehicle view
# --------------

spectator = world.get_spectator()
transform = ego_vehicle.get_transform()
spectator.set_transform(carla.Transform(transform.location + carla.Location(y=-5,z=2.5),
carla.Rotation(yaw=90)))

In [4]:
## creating parking vehicles

position = carla.Transform(ego_vehicle.get_transform().location + carla.Location(-3.5,20,0.5), 
                           carla.Rotation(0,90,0))
parked_vehicle = world.spawn_actor(random.choice(vehicle_blueprints), position)

In [5]:
# --------------
# Spawn pedestrian
# --------------

walkers_blueprints = world.get_blueprint_library().filter('*walker*')
walker_loc = carla.Transform(parked_vehicle.get_transform().location + carla.Location(-3.5,3,0.5), 
                           carla.Rotation(0,90,0))
walker = world.spawn_actor(random.choice(walkers_blueprints), walker_loc)

In [6]:
controller_walker = world.get_blueprint_library().find('controller.ai.walker')


In [9]:
x = world.spawn_actor(controller_walker, walker_loc, attach_to=walker)

In [12]:
world.wait_for_tick()

In [10]:
x.start()

In [11]:
x.set_max_speed(1)

In [13]:
x.go_to_location(world.get_random_location_from_navigation())

: 

In [9]:
controller.start()

In [10]:
controller.go_to_location(carla.Location(5,10,2))

: 

In [130]:
#ego_vehicle.set_autopilot(True)
ego_vehicle.enable_constant_velocity(carla.Vector3D(5,0,0))
#ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))

In [16]:
walker.apply_control(carla.WalkerControl(carla.Vector3D(5.0, 3.0, 0.0), speed=0.5))

In [17]:
## DESTROYING ACTORS vehicles
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

    
ego_cam.destroy()
walker.destroy()

True

In [46]:
print(world.get_actors())

[Actor(id=1, type=spectator), Actor(id=2, type=traffic.stop), Actor(id=3, type=traffic.stop), Actor(id=4, type=traffic.stop), Actor(id=5, type=traffic.stop), Actor(id=6, type=traffic.stop), Actor(id=7, type=traffic.unknown), Actor(id=8, type=traffic.unknown), Actor(id=9, type=traffic.unknown), Actor(id=10, type=traffic.unknown), Actor(id=11, type=traffic.unknown), Actor(id=12, type=traffic.unknown), Actor(id=13, type=traffic.stop), Actor(id=14, type=traffic.stop), Actor(id=15, type=traffic.stop), Actor(id=16, type=traffic.stop), Actor(id=17, type=traffic.stop), Actor(id=18, type=traffic.stop), Actor(id=19, type=traffic.stop), Actor(id=20, type=traffic.stop), Actor(id=21, type=traffic.stop), Actor(id=22, type=traffic.stop), Actor(id=23, type=traffic.stop), Actor(id=24, type=traffic.stop), Actor(id=25, type=traffic.stop), Actor(id=26, type=traffic.stop), Actor(id=27, type=traffic.stop), Actor(id=28, type=traffic.stop), Actor(id=29, type=traffic.stop), Actor(id=30, type=traffic.stop), Act